In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE bronze_fix_messages
COMMENT "The raw fix messages, ingested from a S3 bucket or else."
AS SELECT * FROM cloud_files("dbfs:/Volumes/workspace/dev/trading/fix_messages", "text",
                             map("cloudFiles.inferColumnTypes", "true"))

In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE silver_fix_messages
CONSTRAINT valid_sender EXPECT (SenderCompID IS NOT NULL)
COMMENT "Parsed FIX messages from the bronze layer with constraint on SenderCompID"
AS
SELECT
  value,
  regexp_extract(value, '8=([^|]+)', 1) AS BeginString,
  regexp_extract(value, '35=([^|]+)', 1) AS MsgType,
  regexp_extract(value, '49=([^|]+)', 1) AS SenderCompID,
  regexp_extract(value, '56=([^|]+)', 1) AS TargetCompID,
  regexp_extract(value, '34=([^|]+)', 1) AS MsgSeqNum,
  regexp_extract(value, '52=([^|]+)', 1) AS SendingTime,
  regexp_extract(value, '11=([^|]+)', 1) AS ClOrdID,
  regexp_extract(value, '55=([^|]+)', 1) AS Symbol,
  regexp_extract(value, '54=([^|]+)', 1) AS Side,
  regexp_extract(value, '38=([^|]+)', 1) AS OrderQty,
  regexp_extract(value, '44=([^|]+)', 1) AS Price,
  regexp_extract(value, '10=([^|]+)', 1) AS CheckSum
FROM live.bronze_fix_messages
WHERE value LIKE '8=%'


In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE gold_fix_messages
COMMENT "Cleaned and enriched FIX messages with typed columns and labels"
AS
SELECT DISTINCT
  *,
  CAST(OrderQty AS INT) AS OrderQty,
  CAST(Price AS DOUBLE) AS Price,
  CAST(SendingTime AS TIMESTAMP) AS SendingTime,
  CAST(MsgSeqNum AS INT) AS MsgSeqNum,
  CASE 
    WHEN Side = '1' THEN 'Buy'
    WHEN Side = '2' THEN 'Sell'
    ELSE 'Other'
  END AS SideLabel
FROM live.silver_fix_messages
